In [9]:
import matplotlib
matplotlib.use("Agg")
# import the necessary packages
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import AveragePooling2D
from keras.applications import ResNet50, MobileNetV2, MobileNet, InceptionV3
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Input
from keras.models import Model
from keras.utils import to_categorical
from keras.optimizers import SGD
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
from tqdm import tqdm
from scipy import ndimage
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import warnings
import argparse
import pickle
import cv2
import os

In [10]:
warnings.filterwarnings('ignore',category=FutureWarning)
warnings.filterwarnings('ignore',category=DeprecationWarning)

In [11]:
args = {
	"dataset": "data",
	"model": "model/violence.h5",
    "label-bin": "model/lb.pickle",
    "epochs": 10,
    "plot": "plot.png"
}


In [12]:
# initialize the set of labels from the spots activity dataset we are
# going to train our network on
LABELS = set(["Violence", "NonViolence"])

In [13]:
# grab the list of images in our dataset directory, then initialize
# the list of data (i.e., images) and class images
print('-'*100)
print("[INFO] loading images...")
print('-'*100)
imagePaths = list(paths.list_images(args["dataset"]))
data = []
labels = []

----------------------------------------------------------------------------------------------------
[INFO] loading images...
----------------------------------------------------------------------------------------------------


In [16]:
print(imagePaths)

['data\\NonViolence\\NV_1-0000.jpg', 'data\\NonViolence\\NV_1-0005.jpg', 'data\\NonViolence\\NV_1-0010.jpg', 'data\\NonViolence\\NV_1-0015.jpg', 'data\\NonViolence\\NV_1-0020.jpg', 'data\\NonViolence\\NV_1-0025.jpg', 'data\\NonViolence\\NV_1-0030.jpg', 'data\\NonViolence\\NV_1-0035.jpg', 'data\\NonViolence\\NV_1-0040.jpg', 'data\\NonViolence\\NV_1-0045.jpg', 'data\\NonViolence\\NV_1-0050.jpg', 'data\\NonViolence\\NV_1-0055.jpg', 'data\\NonViolence\\NV_1-0060.jpg', 'data\\NonViolence\\NV_1-0065.jpg', 'data\\NonViolence\\NV_10-0000.jpg', 'data\\NonViolence\\NV_10-0005.jpg', 'data\\NonViolence\\NV_10-0010.jpg', 'data\\NonViolence\\NV_10-0015.jpg', 'data\\NonViolence\\NV_10-0020.jpg', 'data\\NonViolence\\NV_10-0025.jpg', 'data\\NonViolence\\NV_10-0030.jpg', 'data\\NonViolence\\NV_10-0035.jpg', 'data\\NonViolence\\NV_10-0040.jpg', 'data\\NonViolence\\NV_10-0045.jpg', 'data\\NonViolence\\NV_10-0050.jpg', 'data\\NonViolence\\NV_10-0055.jpg', 'data\\NonViolence\\NV_10-0060.jpg', 'data\\NonViol

In [17]:
# loop over the image paths
for imagePath in tqdm(imagePaths[::2]):
    # print("Current imagePath:", imagePath)
    
    # Split the path into directory and file components
    directory, filename = os.path.split(imagePath)
    
    # Extract the class label from the directory
    label = os.path.basename(directory)  # Violence / NonViolence

    # if the label of the current image is not part of the labels
    # you are interested in, then ignore the image
    if label not in LABELS:
        continue

    # load the image, convert it to RGB channel ordering, and resize
    # it to be a fixed 224x224 pixels, ignoring aspect ratio
    image = cv2.imread(imagePath)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (224, 224))

    # update the data and labels lists, respectively
    data.append(image)
    labels.append(label)


100%|██████████| 28922/28922 [06:10<00:00, 77.98it/s] 


In [18]:
# convert the data and labels to NumPy arrays
data = np.array(data)
labels = np.array(labels)

In [19]:
# perform one-hot encoding on the labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

In [20]:
# partition the data into training and testing splits using 75% of
# the data for training and the remaining 25% for testing
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.25, stratify=labels, random_state=42)

In [21]:
# initialize the training data augmentation object
trainAug = ImageDataGenerator(
	rotation_range=30,
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
	fill_mode="nearest")

In [22]:
# initialize the validation/testing data augmentation object (which
# we'll be adding mean subtraction to)
valAug = ImageDataGenerator()

In [23]:
# define the ImageNet mean subtraction (in RGB order) and set the
# the mean subtraction value for each of the data augmentation
# objects
mean = np.array([123.68, 116.779, 103.939], dtype="float32")
trainAug.mean = mean
valAug.mean = mean

In [24]:
# load the ResNet-50 network, ensuring the head FC layer sets are left
# off
baseModel = InceptionV3(weights="imagenet", include_top=False,input_tensor=Input(shape=(224, 224, 3)))

In [26]:
# construct the head of the model that will be placed on top of the
# the base model
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(5, 5))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(512, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(len(lb.classes_), activation="softmax")(headModel)

In [27]:
# place the head FC model on top of the base model (this will become
# the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)

In [28]:
# loop over all layers in the base model and freeze them so they will
# *not* be updated during the training process
model.trainable = True
# for layer in model.layers[:20]:
#     layer.trainable=False
# for layer in model.layers[20:]:
#     layer.trainable=True

In [29]:
# compile our model (this needs to be done after our setting our
# layers to being non-trainable)
print('-'*100)
print("[INFO] compiling model...")
print('-'*100)
initial_learning_rate = 1e-4
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=args["epochs"], decay_rate=0.9, staircase=True
)
opt = tf.keras.optimizers.SGD(learning_rate=lr_schedule, momentum=0.9)

# opt = SGD(lr=1e-4, momentum=0.9, decay=1e-4 / args["epochs"])
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
print(model.summary())

----------------------------------------------------------------------------------------------------
[INFO] compiling model...
----------------------------------------------------------------------------------------------------
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 111, 111, 32)         864       ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 111, 111, 32)         96        ['conv2d[0][0]']              
 Normalization)                                                 

In [30]:
# train the head of the network for a few epochs (all other layers
# are frozen) -- this will allow the new FC layers to start to become
# initialized with actual "learned" values versus pure random
# from scipy import ndimage
# import tensorflow as tf
# from tensorflow.keras.preprocessing.image import ImageDataGenerator

# ... (rest of your imports and code)

print('-'*100)
print("[INFO] training head...")
print('-'*100)
H = model.fit(
    trainAug.flow(trainX, trainY, batch_size=32),
    steps_per_epoch=len(trainX) // 32,
    validation_data=valAug.flow(testX, testY),
    validation_steps=len(testX) // 32,
    epochs=args["epochs"])



----------------------------------------------------------------------------------------------------
[INFO] training head...
----------------------------------------------------------------------------------------------------


C:\Users\pskat\AppData\Local\Temp\ipykernel_8792\3741202086.py:13: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  H = model.fit_generator(


Epoch 1/10


677/677 [==============================] - 3395s 5s/step - loss: 0.6787 - accuracy: 0.5955 - val_loss: 0.6055 - val_accuracy: 0.6837
Epoch 2/10
677/677 [==============================] - 37102s 55s/step - loss: 0.6593 - accuracy: 0.6144 - val_loss: 0.6046 - val_accuracy: 0.6849
Epoch 3/10
677/677 [==============================] - 3108s 5s/step - loss: 0.6589 - accuracy: 0.6180 - val_loss: 0.6052 - val_accuracy: 0.6801
Epoch 4/10
677/677 [==============================] - 3108s 5s/step - loss: 0.6634 - accuracy: 0.6139 - val_loss: 0.6049 - val_accuracy: 0.6860
Epoch 5/10
677/677 [==============================] - 2924s 4s/step - loss: 0.6601 - accuracy: 0.6175 - val_loss: 0.6051 - val_accuracy: 0.6815
Epoch 6/10
677/677 [==============================] - 2900s 4s/step - loss: 0.6576 - accuracy: 0.6155 - val_loss: 0.6061 - val_accuracy: 0.6786
Epoch 7/10
677/677 [==============================] - 3005s 4s/step - loss: 0.6594 - accuracy: 0.6158 - val_loss: 0.6060 - val_accur

In [31]:
# evaluate the network
print('-'*100)
print("[INFO] evaluating network...")
print('-'*100)
predictions = model.predict(testX, batch_size=32)
print(classification_report(testY.argmax(axis=1),
	predictions.argmax(axis=1), target_names=lb.classes_))

----------------------------------------------------------------------------------------------------
[INFO] evaluating network...
----------------------------------------------------------------------------------------------------
226/226 [==============================] - 238s 1s/step
              precision    recall  f1-score   support

 NonViolence       0.69      0.50      0.58      3199
    Violence       0.67      0.82      0.74      4032

    accuracy                           0.68      7231
   macro avg       0.68      0.66      0.66      7231
weighted avg       0.68      0.68      0.67      7231



In [32]:
# plot the training loss and accuracy
print('-'*100)
print("[INFO] plot the training loss and accuracy...")
print('-'*100)
N = args["epochs"]
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_accuracy")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_accuracy")
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig(args["plot"])

----------------------------------------------------------------------------------------------------
[INFO] plot the training loss and accuracy...
----------------------------------------------------------------------------------------------------


In [33]:
# serialize the model to disk
print('-'*100)
print("[INFO] serializing network...")
print('-'*100)
model.save(args["model"] + ".keras")
# model.save(args["model"])


----------------------------------------------------------------------------------------------------
[INFO] serializing network...
----------------------------------------------------------------------------------------------------


d:\Projects\RL\.venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [37]:
# serialize the label binarizer to disk
f = open(args["label-bin"], "wb")
f.write(pickle.dumps(lb))
f.close()